## Get local db

In [27]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv



load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_KEY')

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectordb= Chroma(persist_directory="./jonhWick_db", embedding_function=embeddings, collection_name="doc_jonhWick")

## We are going to do a Naive RAG.

## Remember:

- R -> Retrieval
- A -> Augmented
- G -> Generation

# Retrieval

In [28]:
# Specifying top k
naive_retriever = vectordb.as_retriever(search_kwargs={ "k" : 10})

# Similarity score threshold retrieval
# naive_retriever = db.as_retriever(search_kwargs={"score_threshold": 0.8}, search_type="similarity_score_threshold")

# Maximum marginal relevance retrieval
# naive_retriever = db.as_retriever(search_type="mmr")

# Augmented

In [29]:
from langchain_core.prompts import ChatPromptTemplate

TEMPLATE = """\
You are happy assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

# Generation

In [30]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

## Finally, we are going to create a Rag Chain. For that, we are going to use LCEL (LangChain Expression Language)
If you want to learn more about LCEL, check this good tutorial: https://www.youtube.com/watch?v=O0dUOtOIrfs

### The simplest way to use LCEL

In [31]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

setup_and_retrieval = RunnableParallel({"question": RunnablePassthrough(), "context": naive_retriever })
output_parser = StrOutputParser()


naive_retrieval_chain = setup_and_retrieval | rag_prompt | chat_model | output_parser


naive_retrieval_chain.invoke( "Did people generally like John Wick?")


'Yes, people generally liked John Wick.'

In [32]:
naive_retrieval_chain.invoke( "What are the reviews with a score greater than 7 and say bad things about the movie?")


"I don't know."

### A little more complex to use LCEL

In [33]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

setup_and_retrieval = RunnableParallel({"question": itemgetter("question") |  RunnablePassthrough(), "context": itemgetter("question") |  naive_retriever })
output_parser = StrOutputParser()


naive_retrieval_chain = setup_and_retrieval | rag_prompt | chat_model | output_parser


naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})

'Yes, people generally liked John Wick based on the positive reviews provided.'

### A little X2 more complex to use LCEL

In [34]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

setup_and_retrieval = RunnableParallel({"question": itemgetter("question") |  RunnablePassthrough(), "context": itemgetter("question") }) | RunnablePassthrough.assign(context=itemgetter("context"))
output_parser = StrOutputParser()


naive_retrieval_chain = setup_and_retrieval | rag_prompt | chat_model | output_parser


naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})

'Yes, people generally like John Wick.'

### A little X3 more complex to use LCEL

In [35]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from operator import itemgetter

setup_and_retrieval = RunnableParallel({"question": itemgetter("question") |  RunnablePassthrough(), "context": itemgetter("question") | naive_retriever }) | RunnablePassthrough.assign(context=itemgetter("context"))

naive_retrieval_chain = setup_and_retrieval | {"response": rag_prompt | chat_model, "context": itemgetter("context")}


naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})

{'response': AIMessage(content='Yes, people generally liked John Wick based on the positive reviews provided.', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 3203, 'total_tokens': 3217}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-90f78604-e2e8-465a-8976-dc87d18598fc-0'),
 'context': [Document(page_content=": 9\nReview: At first glance, John Wick sounds like a terrible film on paper but with the slickness of Keanu Reeves' performance as the titular character and the sheer brilliance in its action sequences, this marks the best action film of the year and one of the absolute best in the past decade. Following a brutal home invasion that leaves his beloved dog murdered by thugs from his past, John Wick vows revenge on the ones who have taken what he loves most. Like I said, on paper, this seems like a direct-to-DVD slopfest starring John Cena but as a film with Keanu Reeves as the

In [36]:
naive_retrieval_chain.invoke({"question" : "Make a summary of the reviews that talk about John Wick 3 and have a score higher than 7"})

{'response': AIMessage(content="I don't know.", response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 2541, 'total_tokens': 2546}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-355bd85c-466a-408e-9dca-9c556b44ed88-0'),
 'context': [Document(page_content=': 11\nReview: The overrated "John Wick: Chapter 3 - Parabellum" is an action film that gives the sensation of watching a boring videogame. The plot is only John Wick running and killing the assassins that want the 14 million dollar award for his head. After the initial original deaths, the film becomes tiresome, boring and repetitive. My vote is three.', metadata={'Author': 'claudio_carvalho', 'Movie_Title': 'John Wick 3', 'Rating': 3, 'Review_Date': '1 September 2019', 'Review_Title': ' Gives the Sensation of Watching a Boring Videogame\n', 'Review_Url': '/review/rw5093352/?ref_=tt_urv', 'row': 11, 'source': 'data/john_wick_3.csv'}),
  Do